<a id=0></a>
# 4.Categorical Plot

---
### [1.SearbornのデータセットからDataFrameを作成 ](#1)
### [2.欠損値の処理 ](#2)
### [3.欠損した年齢を考えてみる（プロットの例を示す）](#3)
### [4.カテゴリカルプロット](#4)
---

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

---
<a id=1></a>
[Topへ](#0)

---
## 1. SearbornのデータセットからDataFrameを作成

* データセット(Titanic)を読み込む
* データセットの理解と整理、Kaggleの紹介
* 重複カラムの処理
---

データセット(Titanic)を読み込む

In [ ]:
sns.get_dataset_names()
# このような関数はないというエラー、もしくは空のリストがリターンされるかもしれません
# その場合は続くセルのload_datasetもエラーになると思われます
# ダウンロードいただいている"sns_datasets"内のcsvファイルを読み込んで学習を先へお進めください

In [ ]:
df = sns.load_dataset('titanic')
# エラーの場合はダウンロードいただいている"sns_datasets"内のcsvファイルを読み込んで学習を先へお進めください

In [ ]:
df.head(3)

In [ ]:
df.shape

データセットの理解と整理、Kaggleの紹介

<p>
    <a href="https://www.kaggle.com/competitions/titanic/data" target="_blank" rel="noreferrer noopener">
        Kaggle Dataset Titanic のページへ
    </a>
</p>

    survived : Survival	0 = No, 1 = Yes  
    pclass   : Ticket class	1 = 1st, 2 = 2nd, 3 = 3rd  
    sex	  : Sex	  
    age      : Age in years  	
    sibsp    : # of siblings / spouses aboard the Titanic  (兄弟姉妹 / 配偶者)  
    parch    : # of parents / children aboard the Titanic	(親 / 子)  
    fare     : Passenger fare	
    embarked : Port of Embarkation	C = Cherbourg, Q = Queenstown, S = Southampton  
    who      : man or female or child
    class, who, adult_male, deck, embark_town, alive, alone
    ※ 一部変更しています

重複カラムの処理（'alive', 'class', 'adult_male', 'embark_town', 'alone'）

In [ ]:
df.info()

In [ ]:
# survivedとaliveは同じこと？
df['survived'].unique(), df['alive'].unique()

In [ ]:
df[['survived', 'alive']].value_counts()
# 0 には no のみ、1 には yesのみ。

In [ ]:
df[['pclass', 'class']].value_counts()
# それぞれ1対1で対応している

In [ ]:
df[['embarked', 'embark_town']].value_counts()
# それぞれ1対1で対応している

In [ ]:
df[['who', 'adult_male']].value_counts()
# manであればadult_male。whoで判断可能

In [ ]:
df[(df['sibsp'] == 0) & (df['parch'] == 0)]['alone'].unique()

In [ ]:
df[(df['sibsp'] != 0) | (df['parch'] != 0)]['alone'].unique()

<p style="background-color:#ffd">
    'alive', 'class', 'adult_male', 'embark_town', 'alone' のカラムを削除 
    </p>

In [ ]:
df.drop(['alive', 'class', 'adult_male', 'embark_town', 'alone'], axis=1, inplace=True)

In [ ]:
df.shape

In [ ]:
df.info()

---
<a id=2></a>
[Topへ](#0)

---
## 2. 欠損値の処理

* 欠損値の確認
* 欠損値の処理('deck')
* 欠損値の処理('embarked')：予測を試みる
---

欠損値の確認

In [ ]:
df.isnull().sum()

欠損値の処理('deck')

In [ ]:
df['deck'].value_counts().sort_index()

In [ ]:
df.groupby('deck')['survived'].describe()
# クラス別の差は存在するが、プロットのセクションのためnon-nullが77％を占めるこのカラムは削除する

In [ ]:
df.drop('deck', axis=1, inplace=True)

In [ ]:
df.shape

欠損値の処理('embarked')：予測を試みる

In [ ]:
df[df['embarked'].isnull()]
# 年齢以外の項目が一致している

In [ ]:
# embarkedとsurvivedの関係
df.groupby('embarked')['survived'].describe()
# 平均で確認するとCからの乗船客の生存率が高い

In [ ]:
# 同条件の乗船客がいないかを調べる
df[(df['survived'] == 1) & (df['pclass'] == 1) & (df['sex'] == 'female') & (df['fare'] == 80) & (df['sibsp'] == 0) & (df['parch'] == 0)]

In [ ]:
df[(df['pclass'] == 1) & (df['sex'] == 'female') & (df['fare'] == 80) & (df['sibsp'] == 0) & (df['parch'] == 0)]

In [ ]:
vc1 = df[(df['pclass'] == 1) & (df['sex'] == 'female') & (df['sibsp'] == 0) & (df['parch'] == 0)]['embarked'].value_counts()
vc1

In [ ]:
vc2 = df['embarked'].value_counts()[[1, 0]]
vc2

In [ ]:
vc1 / vc2
# Cからの場合同条件の率が10.7％、Sからは2.2％

In [ ]:
# 乗船地と年齢の関係
df[df['sex'] == 'female'].groupby('embarked')['age'].describe()

In [ ]:
# ともにCと推定して、値を更新する
df.loc[61, 'embarked'] = 'C'
df.loc[829, 'embarked'] = 'C'

In [ ]:
df.isnull().sum()

In [ ]:
df.shape

---
<a id=3></a>
[Topへ](#0)

---
## 3. 欠損した年齢を考えてみる（プロットの例を示す）

* 年齢を確認
* 年齢の分布
    * **sns.histplot()**
    <p>
        <a href="https://seaborn.pydata.org/generated/seaborn.histplot.html" target="_blank" rel="noreferrer noopener">
            Seaborn.histplot のページへ
        </a>
    </p>


* 'age'と'pclass'の関係
* 'age'と'sex'の関係
* 相関関係（correlation）
    * **sns.heatmap()**
    <p>
        <a href="https://seaborn.pydata.org/generated/seaborn.heatmap.html" target="_blank" rel="noreferrer noopener">
            Seaborn.heatmap のページへ
        </a>
    </p>


* 個別の予測は困難であり、Seabornの学習がメインのため、レコードをすべて削除する
* CSVファイルとして保存する

---

年齢を確認

In [ ]:
# 177 / 891  約20％が欠損
df[df['age'].isnull()].head()

In [ ]:
df['who'].unique()

In [ ]:
df[df['age'].isnull()]['who'].unique()
# nullを持つレコードにはchildは含まれない

In [ ]:
df[df['who'] == 'child']['age'].max()
# 少なくとも15歳よりは上の年齢

In [ ]:
df_adult = df[15 < df['age']]
df_adult.sample(5)

年齢の分布

In [ ]:
df_adult['age'].max()
# 15-80の間の数だと推測できる

In [ ]:
# 詳細は後のレクチャーで
sns.histplot(data=df_adult, x='age', bins=13, label='passenger')
plt.title('adultの年齢分布')
plt.legend()

plt.show()
# 日本語表示ができないため警告文が表示される

In [ ]:
df_adult['age'].mean()

In [ ]:
# google colaboratoryの場合
# !pip install japanize-matplotlib

In [ ]:
# google colaboratoryの場合
# import japanize_matplotlib
# japanize_matplotlib.japanize()

# jupyterでもこちらを利用することができるが、以下のset_themeをコース内では用いていきます

In [ ]:
sns.set_theme(font='MS GOTHIC', context='talk', style='darkgrid')
# context : paper, notebook, talk, poster
# style : dark, darkgrid, white, whitegrid, ticks
# 日本語対応は'MS Gothic', 'MS Mincho', 'Meiryo'。他にもあると思われます。

In [ ]:
# 詳細は後のレクチャーで
sns.histplot(data=df_adult, x='age', bins=13, label='passenger')
plt.title('adultの年齢分布')
plt.legend()

plt.show()

'age'と'pclass'の関係

In [ ]:
df_adult.head(3)

In [ ]:
df.groupby('pclass')['fare'].describe()

In [ ]:
# 詳細は後のレクチャーで
plt.figure(figsize=(12, 5))
sns.histplot(data=df_adult, x='age', bins=13, hue='pclass', palette='Set1', multiple='dodge', shrink=0.8, kde=True)
plt.title('adultの年齢分布')

plt.show()

In [ ]:
# figsize : インチ単位
# dpi : ドットパーインチ
# 例）figsize=(10, 5), dpi=100の場合
# 10×100＝1000  5×100＝500
# 1000×500=50万ピクセル（50万画素）

In [ ]:
# 詳細は後のレクチャーで
fig, axes = plt.subplots(2, 1, figsize=(15, 10), tight_layout=True, sharex=True)
sns.histplot(data=df_adult, x='age', bins=13, hue='pclass', multiple='dodge', palette='Set1', shrink=0.9, ax=axes[0])
sns.histplot(data=df_adult, x='age', bins=13, hue='pclass', multiple='fill', palette='Set2', shrink=0.9, ax=axes[1])
axes[1].set_ylabel('Ratio')
axes[0]

fig.suptitle("adultsの年齢分布")

plt.show()

'age'と'sex'の関係

In [ ]:
# 詳細は後のレクチャーで
plt.figure(figsize=(15,5))
sns.histplot(data=df_adult, x='age', bins=13, hue='sex', multiple='dodge', shrink=0.8)
plt.show()

相関関係

In [ ]:
# 相関関係：correlation
# 相関係数：correlation coefficient
# -1 ～ 1の数字に標準化される
df_adult.corr()

In [ ]:
# 詳細は後のレクチャーで
plt.figure(figsize=(8, 6))
sns.heatmap(df_adult.corr(), annot=True, cmap='coolwarm')
plt.title('df_adult 相関関係')
plt.show()

個別の予測は困難であり、Seabornの学習がメインのため、レコードをすべて削除する

In [ ]:
df.dropna(axis=0, inplace=True)

In [ ]:
df.shape

In [ ]:
df.isnull().sum().sum()

CSVファイルとして保存する

In [ ]:
df.to_csv('titanic_non_null.csv', index=False)

---
<a id=4></a>
[Topへ](#0)

---
## 4. カテゴリカルプロット

* キャットプロットとストリッププロット
 * **sns.catplot()**
    <p>
        <a href="https://seaborn.pydata.org/generated/seaborn.catplot.html" target="_blank" rel="noreferrer noopener">
            Seaborn.catplot のページへ
        </a>
    </p>
    

* カウントプロット
    * **sns.countplot()**
    <p>
        <a href="https://seaborn.pydata.org/generated/seaborn.countplot.html" target="_blank" rel="noreferrer noopener">
            Seaborn.countplot のページへ
        </a>
    </p>


* バープロット
 * **sns.barplot()**
    <p>
        <a href="https://seaborn.pydata.org/generated/seaborn.barplot.html" target="_blank" rel="noreferrer noopener">
            Seaborn.barplot のページへ
        </a>
    </p>


* ボックスプロット
 * **sns.boxplot()**
    <p>
        <a href="https://seaborn.pydata.org/generated/seaborn.boxplot.html" target="_blank" rel="noreferrer noopener">
            Seaborn.boxplot のページへ
        </a>
    </p>
    

* バイオリンプロット
 * **sns.violinplot()**
    <p>
        <a href="https://seaborn.pydata.org/generated/seaborn.violinplot.html" target="_blank" rel="noreferrer noopener">
            Seaborn.violinplot のページへ
        </a>
    </p>
    
    
* スウォームプロット
 * **sns.swarmplot()**
    <p>
        <a href="https://seaborn.pydata.org/generated/seaborn.swarmplot.html" target="_blank" rel="noreferrer noopener">
            Seaborn.swarmplot のページへ
        </a>
    </p>
    


---

キャットプロットとストリッププロット

In [ ]:
df.head()

In [ ]:
df.groupby('embarked')['age'].describe()

In [ ]:
sns.catplot(data=df, x='embarked', y='age', height=6, aspect=2)
plt.show()
#  kind : 'strip'(default), 'swarm', 'box', 'violin', 'boxen', 'point', 'bar'

# catplotはカテゴリカルプロットを包括するようなものだと言ってよいと思います。
# 特徴は行・列にカテゴリーを加えたグリッド型の表現ができることです

In [ ]:
sns.catplot(data=df, x='embarked', y='age', hue='sex', height=6, aspect=2)
plt.show()
# hue:色相
# グリッド型を使えるものはheight,aspectでグラフのサイズを変更します

In [ ]:
sns.catplot(data=df, x='embarked', y='age', hue='sex', dodge=True, height=6, aspect=2)
plt.show()
# hueで重なるものはdodge=Trueで分離できます

In [ ]:
sns.catplot(data=df, x='embarked', y='age', height=4, col='pclass', palette='Greens')
plt.show()
# カラムを指定しました

In [ ]:
sns.catplot(data=df, x='embarked', y='age', height=4, col='pclass', row='survived', palette='Reds')
plt.show()
# ローも加えています

In [ ]:
# row, colを設定
sns.catplot(data=df, x='embarked', y='age', height=4, col='pclass', row='sex', hue='survived', palette='Set2')
plt.show()
# hueにsurvivedを設定し、合計４つのカテゴリ、ひとつの数値データを扱っています

In [ ]:
plt.figure(figsize=(6, 3))
sns.stripplot(data=df, x='embarked', y='age', hue='survived', palette='Set2')
plt.show()
# 単独の場合はstripplotを選択
# グラフのサイズ変更はfigsizeで

カウントプロット

In [ ]:
sns.countplot(data=df, x='survived', hue='sex', palette='gnuplot')
plt.show()

In [ ]:
# xticks、目盛りの変更
sns.countplot(data=df, x='survived', hue='embarked', palette='bone')
plt.xticks([0, 1], ['No', 'Yes'])
plt.show()

バープロット

In [ ]:
sns.barplot(data =df, x = 'embarked', y = 'age', palette= 'Set2')
# estimator:デフォルトは平均, np.std, np.median, np.var, np.cov(共分散)
plt.xticks([0, 1, 2], ['Southampton', 'Cherbourg', 'Queenstown'], rotation=60)
plt.show()
# d(=d)

In [ ]:
df.groupby('embarked')['age'].mean()

ボックスプロット

In [ ]:
sns.boxplot(data=df, x='pclass', y='fare', hue='who', palette='Set3')
plt.show()

In [ ]:
# sym=''で外れ値の表示をなくすことができる（symbol）
sns.boxplot(data=df, x='pclass', y='fare', hue='who', palette='Set3', sym='', width=0.5)
plt.legend(loc='upper center', title='Who')
plt.show()

In [ ]:
df.groupby(['pclass', 'who'])['fare'].describe()

バイオリンプロット

In [ ]:
# combination of boxplot and kernel density estimate
# バイオリンの中には中央値、四分位範囲、最大値もしくは最小値もしくは外れ値との境界値が示されている
plt.figure(figsize=(8, 6))
sns.violinplot(data=df, x='sex', y='age', hue='survived', palette='spring')
plt.show()

In [ ]:
plt.figure(figsize=(8, 6))
sns.violinplot(data=df, x='sex', y='age', hue='survived', palette='spring', split=True)
plt.show()

In [ ]:
# inner='quartile'
plt.figure(figsize=(8, 6))
sns.violinplot(data=df, x='sex', y='age', hue='survived', palette='spring', split=True, inner='quartile')
plt.show()

スウォームプロット

In [ ]:
# swarm：群れ
plt.figure(figsize=(10, 6))
sns.swarmplot(data=df, x='sex', y='age', hue='survived', palette='prism')
plt.show()

In [ ]:
# dodge=True
plt.figure(figsize=(10, 6))
sns.swarmplot(data=df, x='sex', y='age', hue='survived', palette='prism', dodge=True, size=4)
plt.legend(loc='upper center', title='Survived')
plt.show()
# 6.7% of the points cannot be placed; you may want to decrease the size of the markers or use stripplot. warnings.warn(msg, UserWarning)に対して, size=4での対応もできるがplt.figure(figsize=(15, 6))としてもよい

---
 <a id=4></a>
[Topへ](#0)

---
## 以上
    
---